In [1]:
%load_ext nb_black

<IPython.core.display.Javascript object>

In [2]:
import os
os.chdir("../..")

<IPython.core.display.Javascript object>

In [3]:
import pandas as pd
import numpy as np

<IPython.core.display.Javascript object>

In [4]:
X = pd.read_parquet("resources/data/clean/X_train.parquet")
y = pd.read_parquet("resources/data/clean/y_train.parquet")

<IPython.core.display.Javascript object>

In [5]:
X.columns

Index(['loan_amnt', 'funded_amnt', 'funded_amnt_inv', 'term', 'int_rate',
       'installment', 'grade', 'emp_title', 'emp_length', 'annual_inc',
       'verification_status', 'purpose', 'dti', 'delinq_2yrs',
       'inq_last_6mths', 'mths_since_last_delinq', 'mths_since_last_record',
       'open_acc', 'pub_rec', 'revol_bal', 'revol_util', 'total_acc',
       'initial_list_status', 'total_pymnt', 'total_pymnt_inv',
       'total_rec_prncp', 'total_rec_int', 'total_rec_late_fee', 'recoveries',
       'collection_recovery_fee', 'last_pymnt_amnt',
       'collections_12_mths_ex_med', 'mths_since_last_major_derog',
       'acc_now_delinq', 'tot_coll_amt', 'tot_cur_bal', 'total_rev_hi_lim',
       'issue_d_month', 'issue_d_year', 'earliest_cr_line_month',
       'earliest_cr_line_year', 'last_pymnt_d_month', 'last_pymnt_d_year',
       'last_credit_pull_d_month', 'last_credit_pull_d_year'],
      dtype='object')

<IPython.core.display.Javascript object>

### Cyclical columns:
* `issue_d_month`
* `earliest_cr_line_month`
* `last_pymnt_d_month`
* `last_credit_pull_d_month`

In [6]:
column = "issue_d_month"

<IPython.core.display.Javascript object>

In [7]:
min_month = 1
max_month = 12
scaled_column = (X[column] - min_month)/max_month*2*np.pi
scaled_column

0         5.235988
1         2.094395
2         0.000000
3         2.617994
4         5.759587
            ...   
177074    5.759587
177075    0.000000
177076    3.141593
177077    0.000000
177078    2.094395
Name: issue_d_month, Length: 177079, dtype: float64

<IPython.core.display.Javascript object>

In [8]:
sine_scaled_column = np.sin(scaled_column)
cosine_scaled_column = np.cos(scaled_column)
data = np.stack([sine_scaled_column, cosine_scaled_column]).T
encoded_column = pd.DataFrame(data=data, columns=[f"{column}_sine", f"{column}_cosine"])
encoded_column

,issue_d_month_sine,issue_d_month_cosine
0,-8.660254e-01,0.500000
1,8.660254e-01,-0.500000
2,0.000000e+00,1.000000
3,5.000000e-01,-0.866025
4,-5.000000e-01,0.866025
...,...,...
177074,-5.000000e-01,0.866025
177075,0.000000e+00,1.000000
177076,1.224647e-16,-1.000000
177077,0.000000e+00,1.000000


<IPython.core.display.Javascript object>

In [10]:
X = pd.concat([X, encoded_column], axis=1).drop(columns=[column])
X

,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,emp_title,emp_length,annual_inc,...,total_rev_hi_lim,issue_d_year,earliest_cr_line_month,earliest_cr_line_year,last_pymnt_d_month,last_pymnt_d_year,last_credit_pull_d_month,last_credit_pull_d_year,issue_d_month_sine,issue_d_month_cosine
0,10825.0,10825.0,10825.0,60 months,22.40,301.45,E,0.176471,10+ years,60528.0,...,10900.0,2013.0,9.0,1999.0,6.0,2015.0,1.0,2016.0,-8.660254e-01,0.500000
1,11325.0,11325.0,11325.0,36 months,15.80,397.04,C,0.000000,< 1 year,34000.0,...,10700.0,2013.0,8.0,2004.0,9.0,2014.0,1.0,2016.0,8.660254e-01,-0.500000
2,8000.0,8000.0,8000.0,36 months,7.90,250.33,A,0.200000,1 year,30000.0,...,23400.0,2014.0,2.0,2001.0,9.0,2015.0,11.0,2015.0,0.000000e+00,1.000000
3,33425.0,33425.0,33400.0,60 months,18.99,866.88,E,0.178864,missing,76000.0,...,49100.0,2014.0,11.0,1993.0,3.0,2015.0,8.0,2015.0,5.000000e-01,-0.866025
4,35000.0,35000.0,35000.0,36 months,10.99,1145.69,B,0.000000,10+ years,110000.0,...,75000.0,2013.0,10.0,1992.0,10.0,2015.0,1.0,2016.0,-5.000000e-01,0.866025
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
177074,5850.0,5850.0,5850.0,36 months,15.20,203.37,D,0.000000,10+ years,23500.0,...,2100.0,2010.0,3.0,2002.0,1.0,2014.0,1.0,2016.0,-5.000000e-01,0.866025
177075,4800.0,4800.0,4800.0,36 months,21.00,180.85,E,0.000000,< 1 year,50000.0,...,5200.0,2013.0,9.0,1996.0,11.0,2015.0,1.0,2016.0,0.000000e+00,1.000000
177076,4800.0,4800.0,4800.0,36 months,5.42,144.77,A,0.000000,1 year,45000.0,...,23800.0,2011.0,4.0,1996.0,8.0,2014.0,8.0,2014.0,1.224647e-16,-1.000000
177077,20050.0,20050.0,20050.0,60 months,21.00,542.42,E,0.000000,10+ years,75000.0,...,28700.0,2013.0,3.0,1997.0,1.0,2015.0,1.0,2016.0,0.000000e+00,1.000000


<IPython.core.display.Javascript object>